In [16]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')
sample_submission = pd.read_csv(r'data\recsys-in-practice\sample_submission.csv')
test_joke_df_nofactrating = pd.read_csv(r'data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

In [22]:
test_joke_df_nofactrating.columns

Index(['UID', 'JID'], dtype='object')

In [3]:
train_joke_df["UID"] = train_joke_df["UID"].astype(str)
train_joke_df["JID"] = train_joke_df["JID"].astype(str)

In [4]:
train_joke_df

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60
...,...,...,...
1448359,22604,26,2.82
1448360,22255,36,-1.94
1448361,21056,40,-9.56
1448362,12328,97,0.87


In [5]:
rows, r_pos = np.unique(train_joke_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(train_joke_df.values[:,1], return_inverse=True)

In [6]:
rows

array(['1', '10', '100', ..., '9997', '9998', '9999'], dtype=object)

In [7]:
cols

array(['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58',
       '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98',
       '99'], dtype=object)

In [8]:
train_df, valid_df, train_queries, valid_queries = train_test_split(train_joke_df, r_pos, test_size=0.1, random_state=42)

In [9]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [10]:
features = ['UID', 'JID']
cat_features = ['UID', 'JID']

In [23]:
train_pool = Pool(train_df[features], label=train_df['Rating'], cat_features=cat_features)
valid_pool = Pool(valid_df[features], label=valid_df['Rating'], cat_features=cat_features)
main_pool = Pool(train_joke_df[features], label=train_joke_df['Rating'], cat_features=cat_features)

test_pool = Pool(test_joke_df_nofactrating[features], cat_features=cat_features)


In [12]:
#train = Pool(
#    data=X_train,
#    label=y_train,
#    group_id=queries_train,
#    cat_features=[0, 1]
#)

#test = Pool(
#    data=X_test,
#    label=y_test,
#    group_id=queries_test,
#    cat_features=[0, 1]
#)

In [14]:
default_parameters = {
    'iterations': 3000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}


In [15]:
model = CatBoostRanker(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\catboost\core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.080339
0:	learn: 5.1425184	test: 5.1462188	best: 5.1462188 (0)	total: 444ms	remaining: 22m 10s
1:	learn: 5.0630290	test: 5.0597954	best: 5.0597954 (1)	total: 692ms	remaining: 17m 17s
2:	learn: 4.9944157	test: 4.9844951	best: 4.9844951 (2)	total: 931ms	remaining: 15m 30s
3:	learn: 4.9351227	test: 4.9188056	best: 4.9188056 (3)	total: 1.04s	remaining: 12m 56s
4:	learn: 4.8828434	test: 4.8616529	best: 4.8616529 (4)	total: 1.15s	remaining: 11m 26s
5:	learn: 4.8377114	test: 4.8120274	best: 4.8120274 (5)	total: 1.26s	remaining: 10m 26s
6:	learn: 4.7988016	test: 4.7687350	best: 4.7687350 (6)	total: 1.36s	remaining: 9m 43s
7:	learn: 4.7649694	test: 4.7303310	best: 4.7303310 (7)	total: 1.47s	remaining: 9m 11s
8:	learn: 4.7348927	test: 4.6967897	best: 4.6967897 (8)	total: 1.58s	remaining: 8m 45s
9:	learn: 4.7084828	test: 4.6666784	best: 4.6666784 (9)	total: 1.69s	remaining: 8m 25s
10:	learn: 4.6856803	test: 4.6405486	best: 4.6405486 (10)	total: 1.8s	remaining: 8m 8s
11:	lea

94:	learn: 4.5196645	test: 4.4315835	best: 4.4315835 (94)	total: 10.9s	remaining: 5m 32s
95:	learn: 4.5196314	test: 4.4315492	best: 4.4315492 (95)	total: 11s	remaining: 5m 32s
96:	learn: 4.5195738	test: 4.4314772	best: 4.4314772 (96)	total: 11.1s	remaining: 5m 31s
97:	learn: 4.5195575	test: 4.4314753	best: 4.4314753 (97)	total: 11.2s	remaining: 5m 31s
98:	learn: 4.5195215	test: 4.4314199	best: 4.4314199 (98)	total: 11.3s	remaining: 5m 31s
99:	learn: 4.5194859	test: 4.4313703	best: 4.4313703 (99)	total: 11.4s	remaining: 5m 31s
100:	learn: 4.5194440	test: 4.4313448	best: 4.4313448 (100)	total: 11.5s	remaining: 5m 30s
101:	learn: 4.5193709	test: 4.4312931	best: 4.4312931 (101)	total: 11.6s	remaining: 5m 30s
102:	learn: 4.5193190	test: 4.4312257	best: 4.4312257 (102)	total: 11.7s	remaining: 5m 29s
103:	learn: 4.5192984	test: 4.4312027	best: 4.4312027 (103)	total: 11.8s	remaining: 5m 29s
104:	learn: 4.5192875	test: 4.4311814	best: 4.4311814 (104)	total: 11.9s	remaining: 5m 29s
105:	learn: 4

185:	learn: 4.5171026	test: 4.4287299	best: 4.4287299 (185)	total: 20.9s	remaining: 5m 16s
186:	learn: 4.5170928	test: 4.4287195	best: 4.4287195 (186)	total: 21s	remaining: 5m 16s
187:	learn: 4.5170720	test: 4.4286945	best: 4.4286945 (187)	total: 21.1s	remaining: 5m 15s
188:	learn: 4.5170632	test: 4.4286812	best: 4.4286812 (188)	total: 21.2s	remaining: 5m 15s
189:	learn: 4.5170565	test: 4.4286802	best: 4.4286802 (189)	total: 21.4s	remaining: 5m 15s
190:	learn: 4.5170471	test: 4.4286712	best: 4.4286712 (190)	total: 21.5s	remaining: 5m 15s
191:	learn: 4.5170272	test: 4.4286473	best: 4.4286473 (191)	total: 21.6s	remaining: 5m 15s
192:	learn: 4.5170219	test: 4.4286349	best: 4.4286349 (192)	total: 21.7s	remaining: 5m 15s
193:	learn: 4.5170094	test: 4.4286328	best: 4.4286328 (193)	total: 21.8s	remaining: 5m 15s
194:	learn: 4.5169922	test: 4.4286136	best: 4.4286136 (194)	total: 21.9s	remaining: 5m 15s
195:	learn: 4.5169650	test: 4.4286022	best: 4.4286022 (195)	total: 22s	remaining: 5m 15s
196

...

2599:	learn: 4.5091802	test: 4.4257980	best: 4.4256961 (1665)	total: 4m 50s	remaining: 44.7s
2600:	learn: 4.5091781	test: 4.4258013	best: 4.4256961 (1665)	total: 4m 50s	remaining: 44.6s
2601:	learn: 4.5091754	test: 4.4258009	best: 4.4256961 (1665)	total: 4m 50s	remaining: 44.5s
2602:	learn: 4.5091745	test: 4.4258027	best: 4.4256961 (1665)	total: 4m 50s	remaining: 44.4s
2603:	learn: 4.5091732	test: 4.4258010	best: 4.4256961 (1665)	total: 4m 51s	remaining: 44.3s
2604:	learn: 4.5091725	test: 4.4258029	best: 4.4256961 (1665)	total: 4m 51s	remaining: 44.2s
2605:	learn: 4.5091710	test: 4.4258030	best: 4.4256961 (1665)	total: 4m 51s	remaining: 44s
2606:	learn: 4.5091698	test: 4.4258028	best: 4.4256961 (1665)	total: 4m 51s	remaining: 43.9s
2607:	learn: 4.5091672	test: 4.4258013	best: 4.4256961 (1665)	total: 4m 51s	remaining: 43.8s
2608:	learn: 4.5091665	test: 4.4258014	best: 4.4256961 (1665)	total: 4m 51s	remaining: 43.7s
2609:	learn: 4.5091644	test: 4.4257987	best: 4.4256961 (1665)	total: 4m 

2688:	learn: 4.5090434	test: 4.4258719	best: 4.4256961 (1665)	total: 5m	remaining: 34.8s
2689:	learn: 4.5090433	test: 4.4258722	best: 4.4256961 (1665)	total: 5m	remaining: 34.6s
2690:	learn: 4.5090425	test: 4.4258743	best: 4.4256961 (1665)	total: 5m	remaining: 34.5s
2691:	learn: 4.5090401	test: 4.4258788	best: 4.4256961 (1665)	total: 5m	remaining: 34.4s
2692:	learn: 4.5090393	test: 4.4258801	best: 4.4256961 (1665)	total: 5m	remaining: 34.3s
2693:	learn: 4.5090375	test: 4.4258800	best: 4.4256961 (1665)	total: 5m 1s	remaining: 34.2s
2694:	learn: 4.5090359	test: 4.4258776	best: 4.4256961 (1665)	total: 5m 1s	remaining: 34.1s
2695:	learn: 4.5090336	test: 4.4258781	best: 4.4256961 (1665)	total: 5m 1s	remaining: 34s
2696:	learn: 4.5090331	test: 4.4258779	best: 4.4256961 (1665)	total: 5m 1s	remaining: 33.9s
2697:	learn: 4.5090320	test: 4.4258780	best: 4.4256961 (1665)	total: 5m 1s	remaining: 33.8s
2698:	learn: 4.5090310	test: 4.4258786	best: 4.4256961 (1665)	total: 5m 1s	remaining: 33.6s
2699:

2779:	learn: 4.5089221	test: 4.4258788	best: 4.4256961 (1665)	total: 5m 10s	remaining: 24.6s
2780:	learn: 4.5089197	test: 4.4258785	best: 4.4256961 (1665)	total: 5m 10s	remaining: 24.5s
2781:	learn: 4.5089193	test: 4.4258789	best: 4.4256961 (1665)	total: 5m 10s	remaining: 24.4s
2782:	learn: 4.5089168	test: 4.4258825	best: 4.4256961 (1665)	total: 5m 11s	remaining: 24.3s
2783:	learn: 4.5089162	test: 4.4258870	best: 4.4256961 (1665)	total: 5m 11s	remaining: 24.1s
2784:	learn: 4.5089148	test: 4.4258919	best: 4.4256961 (1665)	total: 5m 11s	remaining: 24s
2785:	learn: 4.5089123	test: 4.4258896	best: 4.4256961 (1665)	total: 5m 11s	remaining: 23.9s
2786:	learn: 4.5089117	test: 4.4258893	best: 4.4256961 (1665)	total: 5m 11s	remaining: 23.8s
2787:	learn: 4.5089104	test: 4.4258882	best: 4.4256961 (1665)	total: 5m 11s	remaining: 23.7s
2788:	learn: 4.5089094	test: 4.4258911	best: 4.4256961 (1665)	total: 5m 11s	remaining: 23.6s
2789:	learn: 4.5089089	test: 4.4258914	best: 4.4256961 (1665)	total: 5m 

2869:	learn: 4.5087978	test: 4.4259281	best: 4.4256961 (1665)	total: 5m 20s	remaining: 14.5s
2870:	learn: 4.5087973	test: 4.4259291	best: 4.4256961 (1665)	total: 5m 21s	remaining: 14.4s
2871:	learn: 4.5087966	test: 4.4259295	best: 4.4256961 (1665)	total: 5m 21s	remaining: 14.3s
2872:	learn: 4.5087955	test: 4.4259296	best: 4.4256961 (1665)	total: 5m 21s	remaining: 14.2s
2873:	learn: 4.5087919	test: 4.4259305	best: 4.4256961 (1665)	total: 5m 21s	remaining: 14.1s
2874:	learn: 4.5087912	test: 4.4259311	best: 4.4256961 (1665)	total: 5m 21s	remaining: 14s
2875:	learn: 4.5087899	test: 4.4259330	best: 4.4256961 (1665)	total: 5m 21s	remaining: 13.9s
2876:	learn: 4.5087891	test: 4.4259338	best: 4.4256961 (1665)	total: 5m 21s	remaining: 13.8s
2877:	learn: 4.5087883	test: 4.4259323	best: 4.4256961 (1665)	total: 5m 21s	remaining: 13.6s
2878:	learn: 4.5087878	test: 4.4259330	best: 4.4256961 (1665)	total: 5m 22s	remaining: 13.5s
2879:	learn: 4.5087857	test: 4.4259336	best: 4.4256961 (1665)	total: 5m 

2959:	learn: 4.5086788	test: 4.4259481	best: 4.4256961 (1665)	total: 5m 31s	remaining: 4.47s
2960:	learn: 4.5086785	test: 4.4259482	best: 4.4256961 (1665)	total: 5m 31s	remaining: 4.36s
2961:	learn: 4.5086775	test: 4.4259483	best: 4.4256961 (1665)	total: 5m 31s	remaining: 4.25s
2962:	learn: 4.5086766	test: 4.4259480	best: 4.4256961 (1665)	total: 5m 31s	remaining: 4.14s
2963:	learn: 4.5086714	test: 4.4259484	best: 4.4256961 (1665)	total: 5m 31s	remaining: 4.03s
2964:	learn: 4.5086714	test: 4.4259484	best: 4.4256961 (1665)	total: 5m 31s	remaining: 3.92s
2965:	learn: 4.5086698	test: 4.4259478	best: 4.4256961 (1665)	total: 5m 31s	remaining: 3.8s
2966:	learn: 4.5086696	test: 4.4259483	best: 4.4256961 (1665)	total: 5m 31s	remaining: 3.69s
2967:	learn: 4.5086687	test: 4.4259476	best: 4.4256961 (1665)	total: 5m 32s	remaining: 3.58s
2968:	learn: 4.5086672	test: 4.4259475	best: 4.4256961 (1665)	total: 5m 32s	remaining: 3.47s
2969:	learn: 4.5086663	test: 4.4259470	best: 4.4256961 (1665)	total: 5m

In [17]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


4.4256960821322915

In [24]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost0.9.csv')

,Rating
InteractionID,
0,1.348737
1,-2.741850
2,-1.765485
3,5.046078
4,3.770732


In [25]:
parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

model = CatBoostRanker(**parameters)
model.fit(main_pool, eval_set=valid_pool, plot=True)

C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\catboost\core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.104598
0:	learn: 5.1153850	test: 5.1108192	best: 5.1108192 (0)	total: 306ms	remaining: 10m 10s
1:	learn: 5.0169725	test: 4.9991869	best: 4.9991869 (1)	total: 421ms	remaining: 7m
2:	learn: 4.9351641	test: 4.9052967	best: 4.9052967 (2)	total: 668ms	remaining: 7m 24s
3:	learn: 4.8665402	test: 4.8251139	best: 4.8251139 (3)	total: 792ms	remaining: 6m 34s
4:	learn: 4.8101678	test: 4.7604225	best: 4.7604225 (4)	total: 1.05s	remaining: 6m 58s
5:	learn: 4.7615142	test: 4.7018631	best: 4.7018631 (5)	total: 1.18s	remaining: 6m 31s
6:	learn: 4.7218404	test: 4.6536295	best: 4.6536295 (6)	total: 1.3s	remaining: 6m 11s
7:	learn: 4.6885635	test: 4.6130843	best: 4.6130843 (7)	total: 1.42s	remaining: 5m 54s
8:	learn: 4.6617920	test: 4.5800755	best: 4.5800755 (8)	total: 1.54s	remaining: 5m 41s
9:	learn: 4.6386253	test: 4.5504704	best: 4.5504704 (9)	total: 1.66s	remaining: 5m 31s
10:	learn: 4.6195351	test: 4.5258487	best: 4.5258487 (10)	total: 1.79s	remaining: 5m 23s
11:	learn: 4.60

94:	learn: 4.5093854	test: 4.3665878	best: 4.3665878 (94)	total: 11.9s	remaining: 3m 59s
95:	learn: 4.5093486	test: 4.3665316	best: 4.3665316 (95)	total: 12.1s	remaining: 3m 59s
96:	learn: 4.5093366	test: 4.3664927	best: 4.3664927 (96)	total: 12.2s	remaining: 3m 58s
97:	learn: 4.5093126	test: 4.3664823	best: 4.3664823 (97)	total: 12.3s	remaining: 3m 58s
98:	learn: 4.5092965	test: 4.3664501	best: 4.3664501 (98)	total: 12.4s	remaining: 3m 58s
99:	learn: 4.5092679	test: 4.3664226	best: 4.3664226 (99)	total: 12.6s	remaining: 3m 58s
100:	learn: 4.5092418	test: 4.3663775	best: 4.3663775 (100)	total: 12.7s	remaining: 3m 58s
101:	learn: 4.5092212	test: 4.3663342	best: 4.3663342 (101)	total: 12.8s	remaining: 3m 58s
102:	learn: 4.5091936	test: 4.3662820	best: 4.3662820 (102)	total: 12.9s	remaining: 3m 58s
103:	learn: 4.5090836	test: 4.3661440	best: 4.3661440 (103)	total: 13.1s	remaining: 3m 58s
104:	learn: 4.5090753	test: 4.3661231	best: 4.3661231 (104)	total: 13.2s	remaining: 3m 58s
105:	learn:

186:	learn: 4.5070333	test: 4.3641573	best: 4.3641573 (186)	total: 23.3s	remaining: 3m 46s
187:	learn: 4.5070082	test: 4.3641306	best: 4.3641306 (187)	total: 23.4s	remaining: 3m 45s
188:	learn: 4.5069896	test: 4.3641091	best: 4.3641091 (188)	total: 23.5s	remaining: 3m 45s
189:	learn: 4.5069788	test: 4.3641179	best: 4.3641091 (188)	total: 23.7s	remaining: 3m 45s
190:	learn: 4.5069686	test: 4.3641041	best: 4.3641041 (190)	total: 23.8s	remaining: 3m 45s
191:	learn: 4.5069344	test: 4.3640717	best: 4.3640717 (191)	total: 23.9s	remaining: 3m 45s
192:	learn: 4.5069267	test: 4.3640559	best: 4.3640559 (192)	total: 24s	remaining: 3m 44s
193:	learn: 4.5069008	test: 4.3640401	best: 4.3640401 (193)	total: 24.1s	remaining: 3m 44s
194:	learn: 4.5068582	test: 4.3640067	best: 4.3640067 (194)	total: 24.2s	remaining: 3m 44s
195:	learn: 4.5068508	test: 4.3640005	best: 4.3640005 (195)	total: 24.4s	remaining: 3m 44s
196:	learn: 4.5068324	test: 4.3639650	best: 4.3639650 (196)	total: 24.5s	remaining: 3m 44s
1

...

1631:	learn: 4.5009173	test: 4.3609919	best: 4.3609919 (1631)	total: 3m 18s	remaining: 44.7s
1632:	learn: 4.5009171	test: 4.3609922	best: 4.3609919 (1631)	total: 3m 18s	remaining: 44.6s
1633:	learn: 4.5009164	test: 4.3609918	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.4s
1634:	learn: 4.5009158	test: 4.3609929	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.3s
1635:	learn: 4.5009118	test: 4.3609919	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.2s
1636:	learn: 4.5009113	test: 4.3609921	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.1s
1637:	learn: 4.5009103	test: 4.3609922	best: 4.3609918 (1633)	total: 3m 18s	remaining: 43.9s
1638:	learn: 4.5009043	test: 4.3609913	best: 4.3609913 (1638)	total: 3m 18s	remaining: 43.8s
1639:	learn: 4.5009037	test: 4.3609919	best: 4.3609913 (1638)	total: 3m 19s	remaining: 43.7s
1640:	learn: 4.5009031	test: 4.3609878	best: 4.3609878 (1640)	total: 3m 19s	remaining: 43.6s
1641:	learn: 4.5009006	test: 4.3609897	best: 4.3609878 (1640)	total: 3

1721:	learn: 4.5007708	test: 4.3609699	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.8s
1722:	learn: 4.5007686	test: 4.3609693	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.7s
1723:	learn: 4.5007639	test: 4.3609698	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.6s
1724:	learn: 4.5007611	test: 4.3609706	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.5s
1725:	learn: 4.5007594	test: 4.3609701	best: 4.3609654 (1714)	total: 3m 30s	remaining: 33.3s
1726:	learn: 4.5007584	test: 4.3609642	best: 4.3609642 (1726)	total: 3m 30s	remaining: 33.2s
1727:	learn: 4.5007576	test: 4.3609657	best: 4.3609642 (1726)	total: 3m 30s	remaining: 33.1s
1728:	learn: 4.5007544	test: 4.3609637	best: 4.3609637 (1728)	total: 3m 30s	remaining: 33s
1729:	learn: 4.5007534	test: 4.3609616	best: 4.3609616 (1729)	total: 3m 30s	remaining: 32.9s
1730:	learn: 4.5007528	test: 4.3609592	best: 4.3609592 (1730)	total: 3m 30s	remaining: 32.7s
1731:	learn: 4.5007499	test: 4.3609599	best: 4.3609592 (1730)	total: 3m 

1810:	learn: 4.5006177	test: 4.3609105	best: 4.3609105 (1810)	total: 3m 40s	remaining: 23s
1811:	learn: 4.5006158	test: 4.3609136	best: 4.3609105 (1810)	total: 3m 40s	remaining: 22.9s
1812:	learn: 4.5006141	test: 4.3609128	best: 4.3609105 (1810)	total: 3m 40s	remaining: 22.8s
1813:	learn: 4.5006121	test: 4.3609114	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.7s
1814:	learn: 4.5006096	test: 4.3609155	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.5s
1815:	learn: 4.5006037	test: 4.3609149	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.4s
1816:	learn: 4.5006031	test: 4.3609183	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.3s
1817:	learn: 4.5006027	test: 4.3609173	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.2s
1818:	learn: 4.5005992	test: 4.3609177	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.1s
1819:	learn: 4.5005968	test: 4.3609170	best: 4.3609105 (1810)	total: 3m 41s	remaining: 21.9s
1820:	learn: 4.5005912	test: 4.3609149	best: 4.3609105 (1810)	total: 3m 

1900:	learn: 4.5004751	test: 4.3609248	best: 4.3609103 (1836)	total: 3m 51s	remaining: 12.1s
1901:	learn: 4.5004745	test: 4.3609277	best: 4.3609103 (1836)	total: 3m 52s	remaining: 12s
1902:	learn: 4.5004744	test: 4.3609273	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.8s
1903:	learn: 4.5004740	test: 4.3609258	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.7s
1904:	learn: 4.5004724	test: 4.3609241	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.6s
1905:	learn: 4.5004694	test: 4.3609239	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.5s
1906:	learn: 4.5004662	test: 4.3609239	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.3s
1907:	learn: 4.5004645	test: 4.3609259	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.2s
1908:	learn: 4.5004614	test: 4.3609193	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.1s
1909:	learn: 4.5004612	test: 4.3609178	best: 4.3609103 (1836)	total: 3m 53s	remaining: 11s
1910:	learn: 4.5004602	test: 4.3609193	best: 4.3609103 (1836)	total: 3m 53

1989:	learn: 4.5003237	test: 4.3608968	best: 4.3608933 (1980)	total: 4m 2s	remaining: 1.22s
1990:	learn: 4.5003232	test: 4.3608969	best: 4.3608933 (1980)	total: 4m 2s	remaining: 1.1s
1991:	learn: 4.5003229	test: 4.3608978	best: 4.3608933 (1980)	total: 4m 3s	remaining: 976ms
1992:	learn: 4.5003228	test: 4.3608984	best: 4.3608933 (1980)	total: 4m 3s	remaining: 854ms
1993:	learn: 4.5003220	test: 4.3608975	best: 4.3608933 (1980)	total: 4m 3s	remaining: 732ms
1994:	learn: 4.5003212	test: 4.3608934	best: 4.3608933 (1980)	total: 4m 3s	remaining: 610ms
1995:	learn: 4.5003179	test: 4.3608869	best: 4.3608869 (1995)	total: 4m 3s	remaining: 488ms
1996:	learn: 4.5003145	test: 4.3608862	best: 4.3608862 (1996)	total: 4m 3s	remaining: 366ms
1997:	learn: 4.5003127	test: 4.3608901	best: 4.3608862 (1996)	total: 4m 3s	remaining: 244ms
1998:	learn: 4.5003106	test: 4.3608879	best: 4.3608862 (1996)	total: 4m 3s	remaining: 122ms
1999:	learn: 4.5003102	test: 4.3608895	best: 4.3608862 (1996)	total: 4m 4s	remain

In [26]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


4.360886211442052


In [27]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost.csv')

,Rating
InteractionID,
0,1.088325
1,-2.773870
2,-1.315627
3,5.417442
4,4.914472
